In [1]:
import numpy as np
# import PyPDF2
from nltk.corpus import stopwords
import re

In [6]:
def readtxt(path):
    f = open(path)
    text = f.read()
    f.close()
    return text

In [7]:
text = readtxt("./One_Indian_Girl_-_Chetan_Bhagat-Redicals.txt")

In [8]:
print(len(text))
print(text[:500])
type(text)

464927
 

ONE
INDIAN
GIRL

Chetan Bhagat is the author of six bestselling novels—Fl've Point Someone (2004), One Night @ the
Call Center (2005), The 3Mislakes ofMy Life (2008), 2 States (2009), Revolution 2020 (2011) and
HalfGirlfriend (2014)—which have sold over ten million copies and have been translated into over
a dozen languages worldwide.

In 2008, The New York Times called him ‘the biggest—selling author in India’s history’, a
position he has maintained to date. Almost all his books have been ad


str

## Normalise document

In [9]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
tknizer = nltk.WordPunctTokenizer()
stop_wrd = stopwords.words("english")
corpus = nltk.sent_tokenize(text)
print(len(corpus))
corpus[890]

[nltk_data] Downloading package stopwords to /home/puneet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/puneet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


11245


'“Hi, Brij esh.'

In [10]:
def normalise_document(doc):
    doc = re.sub(r"^a-zA-Z\s","",doc,re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    
    words = tknizer.tokenize(doc)
#     print(words[5:10])
    fil_words = [w for w in words if w not in stop_wrd]
    
    doc = " ".join(fil_words)
    return doc

In [11]:
norm_doc = np.vectorize(normalise_document) #it only transfer your functino in numpy for broad casting purpose
doc  = norm_doc(corpus)

In [12]:
doc = [sent for sent in doc if len(sent.split())>2]
print(type(doc))
print(len(doc))
doc[890]

<class 'list'>
10299


'phone buzzed .'

## Corpus Vocubalary

In [13]:
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence

Using TensorFlow backend.
/home/puneet/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [16]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(doc)
word2id = tokenizer.word_index
print(len(word2id.keys()))
text.text_to_word_sequence(doc[520])

6556


['set',
 'routines',
 'protocols',
 'tools',
 'building',
 'software',
 'applications']

In [17]:
word2id["pad"] = 0
id2word = dict([(v,k) for k , v in word2id.items()])
wids = [[word2id[w] for w in text.text_to_word_sequence(d)] for d in doc]

vocab_size = len(word2id)
emb_size = 100
win_size = 1

print(vocab_size)
print(list(word2id.items())[5])
print(list(id2word.items())[5])
print(list(wids)[8])

6556
('like', 6)
(6, 'like')
[1028, 80, 478, 1850, 368, 2391, 1851, 350, 858, 1150, 1852, 3491, 142, 602, 2392, 23, 2387]


## Skip-Gram Generator

In [18]:
from keras.preprocessing.sequence import skipgrams

In [21]:
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in wids]
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id2word[pairs[i][0]], pairs[i][0], 
          id2word[pairs[i][1]], pairs[i][1], 
          labels[i]))

(2 (733), iauren (6252)) -> 0
(someone (128), point (274)) -> 1
(call (45), 2005 (3462)) -> 1
(worldwide (1847), career (335)) -> 0
(' (1523), matching (1672)) -> 0
(six (322), one (7)) -> 1
(point (274), 21 (5202)) -> 0
(ofmy (1306), revolution (3466)) -> 1
(2011 (3468), iet (307)) -> 0
(2011 (3468), hovering (2172)) -> 0


## Model Architecture

In [61]:
from keras.layers import Dense , Reshape ,Embedding
from keras.layers import dot
from keras.models import Sequential
from keras.models import Model

In [43]:
word_model = Sequential()
word_model.add(Embedding(vocab_size,emb_size,input_length=1))
word_model.add(Reshape((emb_size,)))

In [44]:
context_model =Sequential()
context_model.add(Embedding(vocab_size,emb_size,input_length=1))
context_model.add(Reshape((emb_size,)))

In [62]:
model = Sequential()
# model.add(dot([word_model,context_model],axes=))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss = "mean_squared_error",optimizer = "rmsprop")